In [1]:
from missing_bids import auction_data, analytics, environments
hist_plot = auction_data.hist_plot
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
sns.set_style('white')
%matplotlib inline

In [2]:
tsuchiura_data = auction_data.AuctionData('../tests/reference_data/tsuchiura_data.csv')

tsuchiura_before_min_price = auction_data.AuctionData(
    tsuchiura_data.df_bids.loc[tsuchiura_data.data.minprice.isnull()])

In [3]:
filter_ties = auction_data.FilterTies(tolerance=.0001)
filtered_data = filter_ties(tsuchiura_before_min_price)

In [4]:
deviations = [-.02, .0, .001] # [.0, .0008]
demands = [filtered_data.get_counterfactual_demand(rho) for rho in deviations]
print(demands)

[0.657617728531856, 0.21772853185595567, 0.21717451523545706]


In [5]:
constraints = [environments.MarkupConstraint(max_markup=.5, min_markup=.01), 
               environments.InformationConstraint(k=.5, sample_demands=demands)]

In [8]:
# result with seed 0

min_collusion_solver = analytics.MinCollusionIterativeSolver(
    data=tsuchiura_before_min_price, 
    deviations=deviations, 
    metric=analytics.IsNonCompetitive, 
    plausibility_constraints=constraints, 
    num_points=1000.0, 
    seed=0, 
    project=True, 
    filter_ties=None,
    number_iterations=500,
    confidence_level=.95,
    moment_matrix=auction_data.moment_matrix(deviations, 'slope'),
    moment_weights=np.array([0, 0, 1])
)

print('solver demands: {}'.format(min_collusion_solver.demands))
print('rough tolerance T^2: {}'.format(4. / tsuchiura_before_min_price.df_bids.shape[0]))
print('bootstrapped tolerance T^2: {}'.format(min_collusion_solver.tolerance))

result = min_collusion_solver.result
print('minimum share of collusive auctions: {}'.format(result.solution))
result.argmin.head()

solver demands: [0.65899123 0.21792763 0.21491228]
rough tolerance T^2: 0.0021929824561403508
bootstrapped tolerance T^2: 3.7383798380272766e-06
minimum share of collusive auctions: 0.14376307442528846


,-0.02,0.0,0.001,cost,metric,prob
0,0.561254,0.150119,0.144407,0.974354,0.0,8.562369e-01
1,0.657248,0.145698,0.145293,0.755808,1.0,1.437631e-01
2,0.747328,0.145612,0.145205,0.919381,1.0,2.213236e-09
3,0.697393,0.147954,0.147544,0.693926,1.0,1.250389e-09
4,0.623047,0.150945,0.150531,0.694303,1.0,5.522092e-10


In [9]:
# result with seed 1

min_collusion_solver = analytics.MinCollusionIterativeSolver(
    data=tsuchiura_before_min_price, 
    deviations=deviations, 
    metric=analytics.IsNonCompetitive, 
    plausibility_constraints=constraints, 
    num_points=1000.0, 
    seed=2, 
    project=True, 
    filter_ties=None,
    number_iterations=500,
    confidence_level=.95,
    moment_matrix=auction_data.moment_matrix(deviations, 'slope'),
    moment_weights=np.array([0, 0, 1])
)

print('solver demands: {}'.format(min_collusion_solver.demands))
print('rough tolerance T^2: {}'.format(4. / tsuchiura_before_min_price.df_bids.shape[0]))
print('bootstrapped tolerance T^2: {}'.format(min_collusion_solver.tolerance))

result = min_collusion_solver.result
print('minimum share of collusive auctions: {}'.format(result.solution))
result.argmin.head()

solver demands: [0.65899123 0.21792763 0.21491228]
rough tolerance T^2: 0.0021929824561403508
bootstrapped tolerance T^2: 3.7383798380272766e-06
minimum share of collusive auctions: 0.15919273968097608


,-0.02,0.0,0.001,cost,metric,prob
0,0.589483,0.160378,0.154570,0.973108,0.0,8.408073e-01
1,0.639869,0.146355,0.145944,0.743373,1.0,1.591927e-01
2,0.616968,0.148998,0.148584,0.810200,1.0,2.636225e-08
3,0.731394,0.151585,0.151162,0.925161,1.0,4.888917e-10
4,0.598862,0.183548,0.183074,0.756479,1.0,9.008500e-11
